# 4. Экспертные темы

## 4.1. Delta Lake

Delta Lake — это открытый формат хранения данных, который добавляет ACID-транзакции, управление метаданными и другие enterprise-функции к вашим данным в Data Lakes.

| Функции | Преимущество |
| - | - |
| ACID-транзакции | Гарантированная целостность данных при параллельныхзапросах |
| Time Travel | Доступ к историческим данным |
| Schema Enforcement | Контроль схема при записи |
| Upsert/Delete | Поддержка операций MERGE, UPDATE, DELETE |
| Оптимизация файлов | Автоматическая компактификация маленьких файлов |

Установка `pip install delta-spark`

Инициализация Spark с Delta
```
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DeltaLake") \
    .config("spark.jars", "/path/to/delta-core_2.12.jar") \
    .getOrCreate()
```

Создание Delta-таблицы
```
df.write.format("delta").save("/data/delta/events")

# С явным указанием схемы
spark.sql("""
  CREATE TABLE events (
    id LONG,
    date STRING,
    event_type STRING
  ) USING DELTA
  LOCATION '/data/delta/events'
""")
```

Чтение данных
```
delta_df = spark.read.format("delta").load("/data/delta/events")

# Чтение конкретной версии
spark.read.format("delta") \
  .option("versionAsOf", 5) \
  .load("/data/delta/events")
```

#### Time Travel — доступ к истории

Просмотр истории изменений
```
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/data/delta/events")
history = delta_table.history()
history.show()
```

Восстановление данных
```
# Восстановление до версии 2
delta_table.restoreToVersion(2)

# Восстановление до timestamp
delta_table.restoreToTimestamp("2023-01-01 00:00:00")
```

#### Операции Upsert и Delete

MERGE (Upsert)
```
updates_df = spark.createDataFrame([(1, "new_data")], ["id", "data"])

delta_table.alias("target").merge(
    updates_df.alias("updates"),
    "target.id = updates.id"
).whenMatchedUpdate(set={"data": "updates.data"}) \
 .whenNotMatchedInsert(values={
    "id": "updates.id",
    "data": "updates.data"
}).execute()
```

Удаление данных
```
delta_table.delete("date < '2023-01-01'")  # Удалить старые записи
```

#### Оптимизация Delta-таблиц

Компактификация файлов
```
delta_table.optimize().executeCompaction()
```

Z-Ordering (кластеризация)
```
delta_table.optimize().executeZOrderBy("event_type")
```

Вакуумирование (удаление старых версий)
```
delta_table.vacuum()                  # Удалить версии старше 7 дней
delta_table.vacuum(48)                # Удалить версии старше 48 часов
```

#### Практический кейс: CDC (Change Data Capture)

Задача: Реализовать обработку изменений из источника в Delta Lake.
```
# 1. Создаём стриминг из Kafka
changes = spark.readStream \
  .format("kafka") \
  .option("subscribe", "updates") \
  .load()

# 2. Парсим JSON-сообщения
parsed = changes.select(
  from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# 3. Записываем изменения в Delta с MERGE
def upsert_to_delta(microBatchDF, batchId):
    delta_table = DeltaTable.forPath(spark, "/data/delta/target")
    delta_table.alias("t").merge(
        microBatchDF.alias("s"),
        "t.id = s.id"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()

parsed.writeStream \
  .foreachBatch(upsert_to_delta) \
  .option("checkpointLocation", "/checkpoints/delta_cdc") \
  .start()
```

#### Мониторинг и обслуживание

Анализ метаданных
```
spark.sql("ANALYZE TABLE events COMPUTE STATISTICS")
spark.sql("DESCRIBE DETAIL events").show()
```

Размеры файлов
```
spark.sql("""
  SELECT file_size, num_records 
  FROM delta.`/data/delta/events`.files
""").show()
```

## 4.2. MLlib (машинное обучение)

## 4.3. Распределенная обработка (Glue, EMR)

## 4.4. Отладка и логирование  